In [1]:
import os;
%cd ~
%cd mnt
%pwd

import os
os.system('ipcluster start -n 8 --engines=MPIEngineSetLauncher --profile=chase --debug &')

#os.system('sleep 15 &')
import ipyparallel as ipp
rc = ipp.Client(profile='chase');
rc[:].use_cloudpickle()

print('hello from before cpu ');
print(rc.ids)
dview = rc[:]

serial_result = list(map(lambda x:x**10, range(32)))
parallel_result = list(dview.map(lambda x: x**10, range(32)))
print(serial_result)
print(parallel_result, 'parallel_reult')
assert serial_result == parallel_result


/home/jovyan
/home/jovyan/mnt
hello from before cpu 
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 1024, 59049, 1048576, 9765625, 60466176, 282475249, 1073741824, 3486784401, 10000000000, 25937424601, 61917364224, 137858491849, 289254654976, 576650390625, 1099511627776, 2015993900449, 3570467226624, 6131066257801, 10240000000000, 16679880978201, 26559922791424, 41426511213649, 63403380965376, 95367431640625, 141167095653376, 205891132094649, 296196766695424, 420707233300201, 590490000000000, 819628286980801]
[0, 1, 1024, 59049, 1048576, 9765625, 60466176, 282475249, 1073741824, 3486784401, 10000000000, 25937424601, 61917364224, 137858491849, 289254654976, 576650390625, 1099511627776, 2015993900449, 3570467226624, 6131066257801, 10240000000000, 16679880978201, 26559922791424, 41426511213649, 63403380965376, 95367431640625, 141167095653376, 205891132094649, 296196766695424, 420707233300201, 590490000000000, 819628286980801] parallel_reult


In [2]:


import matplotlib as mpl
mpl.use('agg',warn=False)
from matplotlib import pyplot as plt
import time
import pdb
import array
import random
import sys
sys.path.insert(0,"../../../sciunit")

import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
from deap import algorithms
from deap import base
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

import scoop

try:
    import get_neab
except ImportError:
    from neuronunit.tests import get_neab

import quantities as qt
import os
import os.path
from scoop import utils
from scoop import futures

import sciunit.scores as scores
history = tools.History()



init_start=time.time()
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0,
                                                    -1.0, -1.0, -1.0, -1.0))
creator.create("Individual",list, fitness=creator.FitnessMin)

class Individual(object):
    '''
    When instanced the object from this class is used as one unit of chromosome or allele by DEAP.
    Extends list via polymorphism.
    '''
    def __init__(self, *args):
        list.__init__(self, *args)
        self.error=None
        self.results=None
        self.name=''
        self.attrs = {}
        self.params=None
        self.score=None
        self.fitness=None
        self.s_html=None
        self.lookup={}
        self.rheobase=None
toolbox = base.Toolbox()
try:
    #import model_parameters as modelp
    import neuronunit.tests.model_parameters as modelp

except ImportError:
    from neuronunit.tests import model_parameters as params
import model_parameters as modelp
#import os
#os.getcwd()
print(modelp)
BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]

NDIM = len(BOUND_UP) #= len(BOUND_LOW)

import functools

def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("Individual", tools.initIterate, creator.Individual, toolbox.attr_float)
import deap as deap

toolbox.register("population", tools.initRepeat, list, toolbox.Individual)
toolbox.register("select", tools.selNSGA2)

import utilities as outils
model = outils.model
#outils.futures.map = dview.map_sync
outils.map = outils.set_map(dview.map_sync)
map = outils.map
print(outils.map)
print(type(outils.map))
import quantities as pq
import neuronunit.capabilities as cap
import matplotlib.pyplot as plt

def evaluate_e(individual,tuple_params):#This method must be pickle-able for scoop to work.
    '''
    Inputs: An individual gene from the population that has compound parameters, and a tuple iterator that
    is a virtual model object containing an appropriate parameter set, zipped togethor with an appropriate rheobase
    value, that was found in a previous rheobase search.

    outputs: a tuple that is a compound error function that NSGA can act on.

    Assumes rheobase for each individual virtual model object (vms) has already been found
    there should be a check for vms.rheobase, and if not then error.
    Inputs a gene and a virtual model object.
    outputs are error components.
    '''
    gen,vms,rheobase=tuple_params
    assert vms.rheobase == rheobase
    print('{0}'.format(type(vms.rheobase)))
    try:
        assert type(vms.rheobase) is not type(None)

        params = outils.params
        model = outils.model
        print(rheobase,model,params,tuple_params)
        #pdb.set_trace()
        uc = {'amplitude':vms.rheobase}
        current = params.copy()['injected_square_current']
        current.update(uc)
        current = {'injected_square_current':current}
        #Its very important to reset the model here. Such that its vm is new, and does not carry charge from the last simulation
        model.update_run_params(vms.attrs)
        model.inject_square_current(current)
        #reset model, clear away charge from previous model
        model.update_run_params(vms.attrs)
        n_spikes = model.get_spike_count()
        sane = get_neab.suite.tests[0].sanity_check(vms.rheobase*pq.pA,model)

        try:
            assert n_spikes == 1 and sane == True #or n_spikes == 0  # Its possible that no rheobase was found
            for i in [4,5,6]:
                get_neab.suite.tests[i].params['injected_square_current']['amplitude'] = vms.rheobase*pq.pA
            get_neab.suite.tests[0].prediction={}
            assert type(vms.rheobase) != type(None)
            score = get_neab.suite.tests[0].prediction['value']=vms.rheobase * pq.pA
            score = get_neab.suite.judge(model)#passing in model, changes the model
            model.run_number+=1
            error = score.sort_key.values.tolist()[0]
            for i in error:
                if type(i) is type(None):
                    i = np.mean(error)
            #individual.error = error
            individual.rheobase = vms.rheobase
        except:

            inderr = getattr(individual, "error", None)
            if type(inderr) is not (None):
                if len(individual.error)!=0:
                    #the average of 10 and the previous score is chosen as a nominally high distance from zero
                    error = [ (abs(-10.0+i)/2.0) for i in individual.error ]
            else:
                error = [ 100.0 for i in range(0,8) ]

    except Exception as e:
        #raise type(vms.rheobase) is type(None)
        error = [ 100.0 for i in range(0,8) ]

    return error[0],error[1],error[2],error[3],error[4],error[5],error[6],error[7],

#param_dict should be one of scoops shared constants


toolbox.register("evaluate", evaluate_e)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)

toolbox.decorate("mate", history.decorator)
toolbox.decorate("mutate", history.decorator)

toolbox.register("select", tools.selNSGA2)
toolbox.register("map", dview.map_sync)


def plot_ss(vmlist,gen):
    '''
    '''

    import matplotlib.pyplot as plt
    plt.clf()
    for ind,j in enumerate(vmlist):
        if hasattr(ind,'results'):
            plt.plot(ind.results['t'],ind.results['vm'],label=str(vmlist[j].attr) )
            #plt.xlabel(str(vmlist[j].attr))
    plt.savefig('snap_shot_at_gen_'+str(gen)+'.png')
    plt.clf()




def get_trans_dict(param_dict):

    trans_dict = {}
    for i,k in enumerate(list(param_dict.keys())):
        trans_dict[i]=k
    return trans_dict

import model_parameters

param_dict = model_parameters.model_params
#global trans_dict, param_dict

def individual_to_vm(ind,trans_dict=None):
    '''
    This method still not a reliable mapping
    Creates vm.attrs from param_dict and a
    translation dictionary
    '''
    vm = outils.VirtualModel()
    param_dict={}
    #if type(trans_dict) is not type(None):
    if type(trans_dict) is not type(None):

        'transdict is: {0} ind is {1}'.format(trans_dict, ind)
        for i, j in enumerate(ind):
            print(i,j)
            param_dict[trans_dict[i]]=str(j)
        vm.attrs = param_dict
        vm.trans_dict = trans_dict
    #return vm

    if type(trans_dict) is type(None):
        for i, j in enumerate(ind):
            vm.attrs[vm.trans_dict[i]]=str(j)
    return vm


def replace_rh(pop,vmpop):
    '''
    #discard models that cause None rheobase results,
    # and create new models by mutating away from the corresponding  parameters.
    #make sure that the old individual, and virtual model object are
    #over written so do not use list append pattern, as this will not
    #over write the objects in place, but instead just grow the lists inappropriately
    #also some of the lines below may be superflous in terms of machine instructions, but
    #they function to make the code more explicit and human readable.
    '''
    rheobase_checking=outils.rheobase_checking
    from itertools import repeat
    import copy
    for i,ind in enumerate(pop):
        j=0
        while type(vmpop[i].rheobase) is type(None):
            print(j)
            j+=1
            #print('this loop appropriately exits none mutate away from ')
            toolbox.mutate(ind)
            toolbox.mutate(ind)
            toolbox.mutate(ind)
            print('trying mutations: {0}'.format(ind))
            #temp = individual_to_vm(ind,param_dict)
            trans_dict=vmpop[i].trans_dict
            vm_temp = individual_to_vm(ind,trans_dict)
            vmpop[i] = rheobase_checking(vm_temp)
            'trying value {0}'.format(vmpop[i].rheobase)
            ind.rheobase = vmpop[i].rheobase
            pop[i] = ind

            #if type(vmpop[i].rheobase) is not type(None):
            print('rheobase value is updating {0}'.format(vmpop[i].rheobase))
            if type(vmpop[i].rheobase) is not type(None):
                break
        assert type(vmpop[i].rheobase) is not type(None)
    assert ind.rheobase == vmpop[i].rheobase
    assert len(pop)!=0
    assert len(vmpop)!=0


    return pop, vmpop



def test_to_model(vms,local_test_methods):
    import get_neab
    tests = get_neab.suite.tests
    import matplotlib.pyplot as plt
    import copy
    global model
    model.update_run_params(vms.attrs)
    tests = None
    tests = get_neab.suite.tests
    tests[0].prediction={}
    tests[0].prediction['value']=vms.rheobase*qt.pA
    tests[0].params['injected_square_current']['amplitude']=vms.rheobase*qt.pA
    #TODO all of the external rheobase related things need to be re-encapsulated into the NeuroUnit class.
    if local_test_methods in [4,5,6]:
        tests[local_test_methods].params['injected_square_current']['amplitude']=vms.rheobase*qt.pA
    #model.results['vm'] = [ 0 ]
    model.re_init(vms.attrs)
    tests[local_test_methods].generate_prediction(model)



def update_vm_pop(pop,trans_dict,rh_value=None):
    '''
    inputs a population of genes/alleles, the population size MU, and an optional argument of a rheobase value guess
    outputs a population of genes/alleles, a population of individual object shells, ie a pickleable container for gene attributes.
    Rationale, not every gene value will result in a model for which rheobase is found, in which case that gene is discarded, however to
    compensate for losses in gene population size, more gene samples must be tested for a successful return from a rheobase search.
    If the tests return are successful these new sampled individuals are appended to the population, and then their attributes are mapped onto
    corresponding virtual model objects.
    '''
    from itertools import repeat
    import numpy as np
    import copy
    import pdb
    rheobase_checking=outils.rheobase_checking
    vmpop = list(dview.map_sync(individual_to_vm,[toolbox.clone(i) for i in copy.copy(pop)], repeat(trans_dict) ))

    #vmpop = list(futures.map(individual_to_vm,[toolbox.clone(i) for i in copy.copy(pop)], repeat(trans_dict) ))
    vmpop = list(                                                                                                                                   dview.map_sync(rheobase_checking,vmpop))
    print('checkpoint 1 output from parallel map {0}'.format(vmpop))
    rh_value = [ i.rheobase for i in vmpop]
    print(rh_value)

    def rbc(rh_value):
        boolean_r_check=False
        for r in rh_value:
            print(r)
            if type(r) is type(None):
                print(type(r))
                boolean_r_check == True
        return boolean_r_check

    while rbc(rh_value) is True:
        pop,vmpop = replace_rh(pop,vmpop,rh_value)
        rh_value = [ i.rheobase for i in vmpop ]

    rh_value = [ toolbox.clone(i).rheobase for i in copy.copy(vmpop) ]


    assert len(pop) == len(vmpop)
    assert len(pop)!=0
    assert len(vmpop)!=0
    assert rbc(rh_value) is False
    for y,x in enumerate(vmpop):
        assert x.rheobase== rh_value[y]
        print(x.rheobase)
        assert type(x.rheobase) is not type(None)
    return pop,vmpop,rh_value




from scoop import futures, _control, utils, shared



attempting to recover from pickled file
<module 'model_parameters' from '/home/jovyan/mnt/model_parameters.py'>
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
<bound method View.map_sync of <DirectView [0, 1, 2, 3,...]>>
<class 'method'>


In [ ]:

def main():
    global NGEN
    NGEN=3
    global MU
    import numpy as np
    #MU=8#Mu must be some multiple of 4, such that it can be split into even numbers over 8 CPUs
    MU=8
    CXPB = 0.9
    #stats = tools.Statistics(lambda ind: ind.fitness.values)
    pf = tools.ParetoFront()
    from scoop.fallbacks import NotStartedProperly
    trans_dict = get_trans_dict(param_dict)
    if scoop.fallbacks.NotStartedProperly()==False:
        print('this never evaluates {0}'.format(scoop.fallbacks.NotStartedProperly()))

        import pdb; pdb.set_trace()
        shared.setConst(td = trans_dict)
        td = shared.getConst('td')
        print('the shared constant {0}'.format(shared.getConst('td')))
    else:
        td = trans_dict
    pop = toolbox.population(n = MU)
    pop = [toolbox.clone(i) for i in pop]
    #history.update(pop)
    pf.update([toolbox.clone(i) for i in pop])
    #Now attempt to get the rheobase values by first trying the mean rheobase value.
    #This is not an exhaustive search that results in found all rheobase values
    #It is just a trying out an educated guess on each individual in the whole population as a first pass.
    #invalid_ind = [ ind for ind in pop if not ind.fitness.valid ]

    #rheobase_checking = outils.rheobase_checking

    pop,vmpop,rhstorage = list(update_vm_pop(pop,td))
    for y,x in enumerate(vmpop):
        assert x.rheobase== rhstorage[y]
        print(x.rheobase,rhstorage[y])
        #assert type(x.rheobase) is not type(None)
    assert len(pop) != 0
    print(type(pop),type(vmpop))
    assert len(pop) == len(vmpop)
    assert len(vmpop) != 0
    assert len(pop) != 0
    for i in vmpop:
        i.td=shared.getConst('td')
    print('updatevmpop returns a whole heap of nones suggesting its not working {0}'.format(vmpop))
    #population may also be altered in this process.
    pf.update(pop)
    #rhstorage = [ item.rheobase for item in vmpop]

    from itertools import repeat
    #repeat 0, for generation 0
    tuple_storage = zip(repeat(0),vmpop,rhstorage)
    #Now get the fitness of genes:
    #Note the evaluate function called is different
    #pop is being passed attributes or it needs to create them inside evaluate.

    fitnesses = list(dview.map_sync(toolbox.evaluate, pop, tuple_storage))
    invalid_ind = [ ind for ind in pop if not ind.fitness.valid ]
    return fitnesses,invalid_ind


In [ ]:
fitnesses,invalid_ind = main()
print(fitness,invalid_ind)

In [ ]:
ls


In [ ]:
cd mnt

In [ ]:
ls


In [ ]:
cd neuronunit/

In [ ]:
cd neuronunit

In [ ]:
ls


In [ ]:
cd tests

In [ ]:
ls